# Azure AI Agenti s podporo za Model Context Protocol (MCP) - Python

Ta zvezek prikazuje, kako uporabljati Azure AI agente z orodji Model Context Protocol (MCP) v Pythonu. Prikazuje, kako ustvariti inteligentnega agenta, ki lahko izkorišča zunanje MCP strežnike (kot je Microsoft Learn) za izboljšane zmogljivosti z uporabo avtentikacije brez ključev.


## Namestitev potrebnih Python paketov

Najprej moramo namestiti potrebne Python pakete:
- **azure-ai-projects**: Osnovni SDK za Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK za ustvarjanje in upravljanje agentov
- **azure-identity**: Omogoča avtentikacijo brez ključev z uporabo DefaultAzureCredential
- **mcp**: Implementacija protokola Model Context Protocol za Python


## Prednosti avtentikacije brez ključev

Ta zvezek prikazuje **avtentikacijo brez ključev**, ki ponuja več prednosti:
- ✅ **Ni API ključev za upravljanje** - Uporablja avtentikacijo na podlagi identitete Azure
- ✅ **Izboljšana varnost** - Brez shranjevanja skrivnosti v kodi ali konfiguracijskih datotekah
- ✅ **Samodejna rotacija poverilnic** - Azure upravlja življenjski cikel poverilnic
- ✅ **Nadzor dostopa na podlagi vlog** - Uporablja Azure RBAC za natančno določene pravice
- ✅ **Podpora za več okolij** - Deluje brez težav v razvojnem in produkcijskem okolju

`DefaultAzureCredential` samodejno izbere najboljši razpoložljivi vir poverilnic:
1. **Upravljana identiteta** (pri izvajanju v Azure)
2. **Poverilnice Azure CLI** (med lokalnim razvojem)
3. **Poverilnice Visual Studio**
4. **Okoljske spremenljivke** (če so konfigurirane)
5. **Avtentikacija prek interaktivnega brskalnika** (kot zadnja možnost)


## Nastavitev avtentikacije brez ključev

**Predpogoji za avtentikacijo brez ključev:**

### Za lokalni razvoj:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Za Azure okolja:
- Omogočite **sistemsko dodeljeno upravljano identiteto** na vašem Azure viru
- Dodelite ustrezne **RBAC vloge** upravljani identiteti:
  - `Cognitive Services OpenAI User` za dostop do Azure OpenAI
  - `AI Developer` za dostop do projektov Azure AI

### Spremenljivke okolja (neobvezno):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Ni potrebnih API ključev ali povezovalnih nizov!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Uvoz potrebnih knjižnic

Uvozite potrebne Python module:
- **os, time**: Standardne Python knjižnice za okoljske spremenljivke in zamike
- **AIProjectClient**: Glavni odjemalec za Azure AI Projects
- **DefaultAzureCredential**: Brezključna avtentikacija za Azure storitve
- **Razredi, povezani z MCP**: Za ustvarjanje in upravljanje MCP orodij ter obravnavo odobritev


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurirajte nastavitve strežnika MCP

Nastavite konfiguracijo strežnika MCP z uporabo okoljskih spremenljivk in privzetih vrednosti:
- **MCP_SERVER_URL**: URL strežnika MCP (privzeto Microsoft Learn API)
- **MCP_SERVER_LABEL**: Oznaka za identifikacijo strežnika MCP (privzeto "mslearn")

Ta pristop omogoča prilagodljivo konfiguracijo v različnih okoljih.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Ustvarite odjemalca projekta Azure AI (avtentikacija brez ključa)

Inicializirajte odjemalca projekta Azure AI z uporabo **avtentikacije brez ključa**:
- **endpoint**: URL končne točke projekta Azure AI Foundry
- **credential**: Uporablja `DefaultAzureCredential()` za varno avtentikacijo brez ključa
- **Brez potrebnih API ključev**: Samodejno zazna in uporabi najbolj razpoložljive poverilnice

**Potek avtentikacije:**
1. Preveri identiteto, upravljano s strani Azure (v Azure okoljih)
2. Preklopi na poverilnice Azure CLI (za lokalni razvoj)
3. Po potrebi uporabi druge razpoložljive vire poverilnic

Ta pristop odpravlja potrebo po upravljanju API ključev ali povezovalnih nizov v vaši kodi.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Ustvari definicijo orodja MCP

Ustvari orodje MCP, ki se povezuje z Microsoft Learn MCP strežnikom:
- **server_label**: Identifikator za MCP strežnik
- **server_url**: URL končna točka MCP strežnika
- **allowed_tools**: Izbirni seznam za omejitev, katera orodja se lahko uporabljajo (prazni seznam omogoča vsa orodja)

To orodje bo agentu omogočilo dostop do dokumentacije in virov Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Ustvarjanje agenta in izvedba pogovora (brez ključev)

Ta obsežen razdelek prikazuje celoten **potek dela agenta brez ključev**:

1. **Ustvarite AI agenta**: Nastavite agenta z modelom GPT-4.1 nano in orodji MCP
2. **Ustvarite nit**: Vzpostavite pogovorno nit za komunikacijo
3. **Pošljite sporočilo**: Vprašajte agenta o razlikah med Azure OpenAI in OpenAI
4. **Upravljajte odobritve orodij**: Samodejno odobrite klice orodij MCP, ko je to potrebno
5. **Spremljajte izvedbo**: Spremljajte napredek agenta in upravljajte potrebne ukrepe
6. **Prikažite rezultate**: Prikaz pogovora in podrobnosti uporabe orodij

**Funkcije brez ključev:**
- ✅ **Brez vgrajenih skrivnosti** - Vsa avtentikacija se izvaja prek identitete Azure
- ✅ **Privzeto varno** - Uporablja nadzor dostopa na podlagi vlog
- ✅ **Poenostavljena uvedba** - Brez potrebe po upravljanju poverilnic
- ✅ **Prijazno za revizije** - Vsi dostopi so sledljivi prek identitete Azure

Agent bo uporabljal orodja MCP za dostop do virov Microsoft Learn z največjo varnostjo in brez upravljanja API ključev.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
